In [50]:
import pandas as pd
import numpy as np
import os
import pymorphy2
import re
import collections
from nltk.stem.snowball import RussianStemmer
from nltk.corpus import stopwords
from tqdm import tqdm
from collections import Counter

In [51]:
morph = pymorphy2.MorphAnalyzer()
stemmer = RussianStemmer()
russian_stopwords = stopwords.words("russian")

In [52]:
filename = 'data/old/vtb-encurance.csv';
data = pd.read_csv(filename, encoding='utf-8')
data = data[['raw_text']]

In [38]:
data = pd.read_csv("data/retail.csv", encoding='utf-8')
data.drop("Unnamed: 0", inplace=True, axis=1)
data['h'] = data['lemmas_lexic'].map(lambda x: len(x.split(" ")))

In [39]:
data = data[data['h'] > 2]

In [53]:
len(data)

1253

In [41]:
data = data[['raw_text']]

In [54]:
def remove_punct(line):
    line2 = re.sub(r'\d|[\?\.\_\,\:\;\^\$\#\%\@\&\(\)\*\!\<\>\"\'\{\}\[\]\~\+\-\=\/ñ|à|é|ó|ß|á|â|ä|ç|è|é|ê|ì|í|î|ô|ö|ù|ú|ü|œ|ё|ґ|ṗ|‒|“|”|„|き|げ|し|ち|て|上|持|放|\'|\`|\…|\—|\–|\‘|\’\¦\«\°\´\»\×]', ' ', line)
    return line2.replace("  ", " ").lower()

In [55]:
data['text_p'] = data['raw_text'].map(lambda x: remove_punct(x))

In [56]:
data['lemmas']= data['text_p'].map(lambda x: ' '.join([morph.parse(t)[0].normal_form for t in str(x).split(" ")]))
data['stems']= data['text_p'].map(lambda x: ' '.join([stemmer.stem(t) for t in x.split(" ")]))

In [57]:
servisepos_str = "ли что у и в к что так как на по с со до как об но о или от бы из ко за из-за из-под под ввиду в виду насчет насчёт нащет нащёт без для над пред при про у через также так же то как-то как то а будто же да но если зато за то как либо лишь однако ни пока притом причем причём так что такчто хотя чтоб чтобы чем разве неужели ну ведь аж только вовсе все-таки всё-таки таки после"
servisepos = list(set(servisepos_str.split(" ") + russian_stopwords))
data['text_p_servicepos_removed'] = data['text_p'].map(lambda x: ' '.join([t for t in x.split(" ") if t not in servisepos]))
data['stems_servicepos_removed'] = data['stems'].map(lambda x: ' '.join([t for t in x.split(" ") if t not in servisepos]))
data['lemms_servicepos_removed'] = data['lemmas'].map(lambda x: ' '.join([t for t in x.split(" ") if t not in servisepos]))

In [58]:
to_remove = "здравствуйте привет добрый доброе сейчас день вечер ночь пожалуйста было быть был будет будут подскажите утро суток могу можно нужно хотим хочу хочется имеется имеет является являются есть надо мне меня я мы его ему ей ее потом нее якобы мой моя мои мое моё её тоже себя себе нам им нам она он они оно это то вам вас знала знает знают знали узнали узнают сказала сказал сказали скажут уже все всё их ваш наш вы так как это"
remove_lexic = list(set(to_remove.split(" "))) 
data['remove_lexic_raw'] = data['text_p_servicepos_removed'].map(lambda x: ' '.join([t for t in x.split(" ") if t not in remove_lexic]))
data['stems_lexic']= data['remove_lexic_raw'].map(lambda x: ' '.join([stemmer.stem(t) for t in x.split(" ")]))
data['lemmas_lexic']= data['remove_lexic_raw'].map(lambda x: ' '.join([morph.parse(t)[0].normal_form for t in str(x).split(" ")]))

In [59]:
for col in data.columns:
    data[col] = data[col].map(lambda x: x.strip())

In [60]:
data = data[data['stems_lexic'] != ""]
#data = data[data['stems_lexic'] != " "]

In [61]:
data.to_csv("data/ensurance.csv", encoding='utf-8')

In [62]:
data.head()

,raw_text,text_p,lemmas,stems,text_p_servicepos_removed,stems_servicepos_removed,lemms_servicepos_removed,remove_lexic_raw,stems_lexic,lemmas_lexic
0,скажите в какой срок можно отказаться от страх...,скажите в какой срок можно отказаться от страх...,сказать в какой срок можно отказаться от страх...,скаж в как срок можн отказа от страховк по кре...,скажите срок отказаться страховки кредиту почт...,скаж срок можн отказа страховк кредит почт банк,сказать срок отказаться страховка кредит почта...,скажите срок отказаться страховки кредиту почт...,скаж срок отказа страховк кредит почт банк,сказать срок отказаться страховка кредит почта...
1,здравствуйте! купила в прошлом году планшет до...,здравствуйте купила в прошлом году планшет доч...,здравствовать купить в прошлое год планшет доч...,здравств куп в прошл год планшет дочк застрахо...,здравствуйте купила прошлом году планшет дочке...,здравств куп прошл год планшет дочк застрахова...,здравствовать купить прошлое год планшет дочка...,купила прошлом году планшет дочке застраховала...,куп прошл год планшет дочк застрахова втб стра...,купить прошлое год планшет дочка застраховать ...
2,как мне вернуть деньги за страхование которые ...,как мне вернуть деньги за страхование которые ...,как я вернуть деньга за страхование который я ...,как мне вернут деньг за страхован котор мне на...,вернуть деньги страхование которые навязал бан...,вернут деньг страхован котор навяза банк когд ...,вернуть деньга страхование который навязать ба...,вернуть деньги страхование которые навязал бан...,вернут деньг страхован котор навяза банк брал ...,вернуть деньга страхование который навязать ба...
3,"взяла кредит в почта банк, ими же была навязан...",взяла кредит в почта банк ими же была навязана...,взять кредит в почта банк они же быть навязать...,взял кред в почт банк им же был навяза ваш стр...,взяла кредит почта банк ими навязана ваша стра...,взял кред почт банк навяза ваш страховк дне е...,взять кредит почта банк навязать ваш страховка...,взяла кредит почта банк ими навязана ваша стра...,взял кред почт банк им навяза ваш страховк дн...,взять кредит почта банк они навязать ваш страх...
4,", претензию писала еще в октябре. может еще на...",претензию писала еще в октябре может еще написать,претензия писать ещё в октябрь мочь ещё написать,претенз писа ещ в октябр может ещ написа,претензию писала октябре написать,претенз писа ещ октябр ещ написа,претензия писать ещё октябрь мочь ещё написать,претензию писала октябре написать,претенз писа октябр написа,претензия писать октябрь написать
